![trainers in a store](trainers.jpg)

Sports clothing and athleisure attire is a huge industry, worth approximately [$193 billion in 2021](https://www.statista.com/statistics/254489/total-revenue-of-the-global-sports-apparel-market/) with a strong growth forecast over the next decade! 

In this notebook, you will undertake the role of a product analyst for an online sports clothing company. The company is specifically interested in how it can improve revenue. You will dive into product data such as pricing, reviews, descriptions, and ratings, as well as revenue and website traffic, to produce recommendations for its marketing and sales teams.  

You've been provided with four datasets to investigate:

#  brands.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `brand` | Brand of the product | 

# finance.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `listing_price` | Original price of the product | 
| `sale_price` | Discounted price of the product |
| `discount` | Discount off the listing price, as a decimal | 
| `revenue` | Revenue generated by the product |

# info.csv

| Columns | Description |
|---------|-------------|
| `product_name` | Name of the product | 
| `product_id` | Unique product identifier |
| `description` | Description of the product |

# reviews.csv

| Columns | Description |
|---------|-------------|
| `product_id` | Unique product identifier |
| `rating` | Average product rating | 
| `reviews` | Number of reviews for the product |

In [249]:
import pandas as pd

brands = pd.read_csv("brands.csv") 
finance = pd.read_csv("finance.csv")
info = pd.read_csv("info.csv")
reviews = pd.read_csv("reviews.csv")

# Start coding here...

In [250]:
print(brands.info())
print(finance.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3179 entries, 0 to 3178
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   product_id  3179 non-null   object
 1   brand       3120 non-null   object
dtypes: object(2)
memory usage: 49.8+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3179 entries, 0 to 3178
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     3179 non-null   object 
 1   listing_price  3120 non-null   float64
 2   sale_price     3120 non-null   float64
 3   discount       3120 non-null   float64
 4   revenue        3120 non-null   float64
dtypes: float64(4), object(1)
memory usage: 124.3+ KB
None


In [251]:
merged_df = info.merge(finance, on="product_id")
merged_df = merged_df.merge(reviews, on="product_id")
merged_df = merged_df.merge(brands, on="product_id")
merged_df.dropna(inplace=True)

## What is the volume of products and average revenue for Adidas and Nike products based on listing price quartiles?


In [252]:
q=4
labels=['Budget', 'Average', 'Expensive', 'Elite']

merged_df['price_label'] = pd.qcut(merged_df['listing_price'] , q=4, labels=labels)

print(merged_df.head())

                                        product_name  ... price_label
1               Women's adidas Originals Sleek Shoes  ...   Expensive
2                  Women's adidas Swim Puka Slippers  ...      Budget
3   Women's adidas Sport Inspired Questar Ride Shoes  ...   Expensive
4           Women's adidas Originals Taekwondo Shoes  ...   Expensive
5  Women's adidas Sport Inspired Duramo Lite 2.0 ...  ...     Average

[5 rows x 11 columns]


In [253]:
import numpy as np

adidas_vs_nike = merged_df.groupby(['brand', 'price_label'], as_index=False).agg(
    num_products=('price_label', 'count'), 
    mean_revenue=('revenue', 'mean')
).round(2)

In [254]:
print(adidas_vs_nike)

    brand price_label  num_products  mean_revenue
0  Adidas      Budget           574       2015.68
1  Adidas     Average           655       3035.30
2  Adidas   Expensive           759       4621.56
3  Adidas       Elite           587       8302.78
4    Nike      Budget           357       1596.33
5    Nike     Average             8        675.59
6    Nike   Expensive            47        500.56
7    Nike       Elite           130       1367.45


## Do any differences exist between the word count of a product's description and its mean rating?

In [255]:
merged_df['description_length'] = merged_df['description'].str.len()

In [256]:
limits = [0, 100, 200, 300, 400, 500, 600, 700]
labels = ['100', '200', '300', '400', '500', '600', '700']

In [257]:
merged_df['description_length'] = pd.cut(merged_df.description_length, bins= limits, labels=labels)

In [258]:
merged_df['description_length'].value_counts()

300    1785
400     651
200     526
500     118
600      15
700      15
100       7
Name: description_length, dtype: int64

In [259]:
description_lengths = merged_df.groupby('description_length', as_index=False).agg(mean_rating = ('rating', 'mean'), num_reviews = ('reviews', 'count')).round(2)
print(description_lengths)

  description_length  mean_rating  num_reviews
0                100         2.26            7
1                200         3.19          526
2                300         3.28         1785
3                400         3.29          651
4                500         3.35          118
5                600         3.12           15
6                700         3.65           15


## How does the volume of products and median revenue vary between clothing and footwear?

In [260]:
info_finance = info.merge(finance, on='product_id')
print(info_finance.head())

                                       product_name  ...  revenue
0                                               NaN  ...      NaN
1              Women's adidas Originals Sleek Shoes  ...  1641.17
2                 Women's adidas Swim Puka Slippers  ...   398.93
3  Women's adidas Sport Inspired Questar Ride Shoes  ...  2204.37
4          Women's adidas Originals Taekwondo Shoes  ...  5182.70

[5 rows x 7 columns]


In [261]:
info_finance = info_finance.dropna()

In [262]:
info_finance.shape

(3117, 7)

In [263]:
info_finance_filtered = merged_df[merged_df['description'].str.contains('shoe*|trainer*|foot*')]
print(info_finance_filtered.shape)

(2639, 12)


In [264]:
info_finance_clothes = merged_df[~merged_df.isin(info_finance_filtered)]

In [265]:
info_finance_clothes = info_finance_clothes.dropna()
info_finance_clothes.shape

(478, 12)

In [266]:
product_types = pd.DataFrame({
    'num_clothing_products': len(info_finance_clothes['description']),
    'median_clothing_revenue': info_finance_clothes['revenue'].median(),
    'num_footwear_products': len(info_finance_filtered['description']),
    'median_footwear_revenue': info_finance_filtered['revenue'].median()
}, index=[0])

print(product_types)

   num_clothing_products  ...  median_footwear_revenue
0                    478  ...                   3073.3

[1 rows x 4 columns]
